# 5. 오차 역전파(Back Propagation)

## 5.1.1 계산 그래프  
계산 과정을 그래프로 나타내자.  
계산을 정방향으로 하게 되면 순전파, 역방향으로 하면 역전파

## 5.1.2 국소적 계산  
한 노드에서의 계산을 하기 때문에 간단한 계산을 모아 복잡한 계산가능  
이는 Deep learning에서의 연산과 유사하다.

## 5.1.3 왜 계산 그래프로 푸는가?  
위와 같은 계산 그래프를 사용하는 이유는 어느 한 지점(노드)에서의 변화율을 구할 수 있음. 역전파의 기본  
- 궁금한 점   
    그러면 순전파와 동시에 기울기를 구할 수 있어서 효율적? 

## 5.2 연쇄법칙(chain rule)  
$z = \left(x + y\right)^2$ 이 함수는 아래와 같이 합성함수로 구성 가능하다.  
$z = t^2$  
$t = x + y$  
  
합성함수의 미분으로 설명하자면 위의 함수를 미분하면 아래와 같고  
### **${\delta z \over \delta x} = {\delta z \over \delta t}{\delta t \over \delta x}$**  
이를 약분함으로써 아래와 같은 값을 구할 수 있다.  
### **${\delta z \over \delta x}$**  
또한 맨 위의 식을 각각 미분해보자면 아래와 같고,  
### ${\delta z \over \delta t} = 2t, {\delta t \over \delta x} = 1$  
최종적으로는 아래 식과 같이 된다.  
### ${\delta z \over \delta x} = {\delta z \over \delta t}{\delta t \over \delta x} = 2t \times 1 = 2\left(x + y\right)$  
이를 이용하여 계산 그래프에서 각 노드에서의 미분값을 구할 수 있다.

## 5.4.1 곱셈 계층, 덧셈 계층  
모든 계층은 forward(), backword()라는 공통 메서드를 갖도록 구현  
- 그럼 이 방식은 autograd와 다른 방식의 미분방식인지?

In [2]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
    
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y

        return out
    
    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x
        
        return dx, dy

In [3]:
class AddLayer:
    def __init__(self):
        pass
    
    def forward(self):
        out = x + y
        return out
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

## 5.5.1 ReLU Layer  
  
## $y = max(0,x)$
 ${\delta y \over \delta x} = 1~ if~ x > 0$  
$~~~~~~~~~~0~ else$

In [4]:
class Relu:
    def __init__(self):
        self.mask = None
        
    def forawrd(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        return dx

In [5]:
a = np.arange(-5,5)
mask = a<=0 # mask는 0보다 작은 원소를 false로 큰 원소는 true
a[mask] = 0
a

array([0, 0, 0, 0, 0, 0, 1, 2, 3, 4])

## 5.5.2 Sigmoid Layer  
## $y = {1 \over 1~+~exp\left(-x\right)}$  
계산 그래프에서 보자면 계산 순서는 아래와 같다.  
  

|$f(x)$|$f'(x)$|  
|------|-------|
|$x \times -1$|$-1$|    
|$exp(x)$|$exp(x)$|  
|$x + 1$|$1$|  
|$1~/~x$|${1\over - {x^2}}$|  
  
이때 마지막의 $f'(x)$는 역전파 시 y를 받기 때문에 $-y^2$로 설정   

위의 $f(x)$ 는 정방향, $f'(x)$는 역전파로 이루어진다.  
$f'(x)$를 역으로 곱해 나감으로 ${\delta L \over \delta y} = y(1~-~y)$를 알 수 있다.

- 궁금한 점   
    Sigmoid.backward 연산시 dout과 self.out의 차이는?

In [6]:
class Sigmoid:
    def __init__(self):
        self.out = None
    
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out
    
    def backward(self, dout):
        dx = dout * (1 - self.out) * self.out
        return dx

## Affine Layer  
행렬 곱 연산  
흔히 보는 Dense Layer($W \cdot X + B$)와 유사?   
  
* * *  
정방향 연산  
$X(2,) \cdot W(2,3) + B(3,)$  
  
* * *  
역방향 연산(배치 X)  
### ${\delta L \over \delta X}(2,) = {\delta L \over \delta Y}(3,) \cdot W^T(3,2)$  
### ${\delta L \over \delta W}(2,3) = X^T(2,1) \cdot {\delta L \over \delta Y}(1,3)$  
  
* * *
역방향 연산(배치 O)  
### ${\delta L \over \delta X}(N,2) = {\delta L \over \delta Y}(N,3) \cdot W^T(3,2)$  
### ${\delta L \over \delta W}(2,3) = X^T(2,N) \cdot {\delta L \over \delta Y}(N,3)$  

In [7]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
        
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        
        return out
    
    def backword(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        return 0